# TP3


## Introduction
Dans ce Jupyter Notebook, je n'ai pas fourni entièrement le code, les codes ne sont donc pas fonctionnels, mais tous les fichiers sources sont disponibles dans le dossier _code_, cela permet de rendre le jupyter plus lisible.  

## Utilisation
Pour pouvoir relancer les expérimentations des questions 1 à 4, utiliser la commande dans le dossier racine:  
```
python main.py --experiment [LR, NN, L2, BM, NM]
    "LR": logistic regression model
    "NN": one hidden layer model
    "L2": l2 reg on one depth neural network
    "NM": noisy model
    "BM": best model found by reinforcement learning
```

## Question 1. Implémentation de la régression logistique

__Code entier: code/logistic_regression.py__

Dans la première question, il fallait implémenter une régression logistique classique. Pour cela j'ai utilisé la librairie Tensorflow. Je ne vais pas mettre tout le code (dans logistic_regression.py), mais voici les principales parties du code:

### Le modèle de propagation

In [ ]:
# Un vecteur de dimension (batch_size, 784)    
x = self.x

# Un vecteur de poids initialisé avec Xavier init
self.w = tf.get_variable("W", initializer=tf.contrib.layers.xavier_initializer(),
    shape=(self.input_size, self.nb_targets))

# Le biais
self.b = tf.get_variable("b", initializer=tf.contrib.layers.xavier_initializer(), shape=self.nb_targets)

# y = xW + b
self.logits = tf.matmul(self.x, self.w) + self.b

# o_k = softmax(y_k) pour k=1.10
self.probability = tf.nn.softmax(self.logits)

# prediction: Retourne la classe prédite
self.prediction = tf.argmax(self.probability, axis=1)


### Fonction de cout et calcul de gradient

In [ ]:
# Pour l'optimisation, il faut récuperer tous les poids entrainables, calculer leur gradient 
# par rapport à une fonction de cout avec l'algorithme de rétropropagation, puis mettre
# à jour les poids en les fesant pointer dans la direction opposé du gradient.

# Definition de la fonction de cout
# Un petit epsilon est ajouté pour la stabilité numérique
self.loss = tf.reduce_mean(-tf.log(self.probability + 1e-4) * self.y)

# Définition de l'accuracy
self.accuracy = tf.reduce_mean(
    tf.cast(tf.equal(tf.argmax(self.y, 1), self.prediction), tf.float32))

# Récuperer tous les poids du réseau
train_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

# Calculer les gradients par backpropagation
grads = self.optimizer.compute_gradients(loss=self.loss, var_list=train_variables)
# Implementer l'opération de mise à jour des poids
self.train_dis = self.optimizer.apply_gradients(grads, global_step=self.global_step)


Je n'ai pas mis tout le code, mais celui ci est disponible dans le fichier _logisticregression.py_. Après avoir essayé différentes tailles de batches, différents techniques d'optimisation, j'ai décidé d'utiliser la méthode d'__Adam__, réputé pour sa stabilité de convergence, 20 époques, et une taille de batch de __100__.  
Voici les résultats obtenues:  
#### Accuracy
* L'accuracy test est autour de 0.91  
![Accuracy test](images/logistic_regression_mean_accuracy_test.png)
* L'accuracy de validation ne cesse de croitre jusqu'à se stabiliser autour 0.92
![Accuracy validation](images/logistic_regression_mean_accuracy_val)
* L'accuracy sur l'ensemble d'entrainement atteint très vite une valeur plafond (autour de 0.91), avant d'osciller autour de cette valeur. On atteint le maximum que l'on avait obtenu au TP précédent. A noter que cette courbe n'est pas en fonction du nombre d'époque mais en fonction de nombre d'itérations dans chaque époque, donc il y a plus de valeurs que pour les deux autres courbes (test et validation) 
![Accuracy train](images/logistic_regression_accuracy_train.png)

#### Cross entropy
* L'enthropie croisée de l'ensemble de test:
![Accuracy test](images/logistic_regression_loss_test.png)
* L'enthropie croisée de l'ensemble de validation:
![Accuracy validation](images/logistic_regression_lossval.png)
* L'enthropie croisée de l'ensemble d'entrainement:
![Accuracy entrainement](images/logistic_regression_loss_train.png)
Les valeurs des trois fonctions de cout ne divergent pas énormément les unes des autres, le modèle n'introduit pas une grande variance.

#### Remarque
Dans la suite du TP, je vais me focaliser uniquement sur la précision, car c'est ca qui nous intéresse dans notre modèle (c'est le but), tandis que la fonction d'enthropie croisée est le _moyen_ (fonction différentiable).

## Question2. Implémentation du modèle avec une couche cachée

__Code entier: code/onehiddenlayer.py__

Comparé au code de la question 1, la seule partie qui change est

In [ ]:
x = self.x

# Hidden layer
w1 = tf.get_variable("W1", initializer=tf.contrib.layers.xavier_initializer(),
    shape=(self.input_size, self.input_size / 2))
b1 = tf.get_variable("b1", initializer=tf.contrib.layers.xavier_initializer(), shape=self.input_size / 2)
self.h1 = tf.nn.relu(tf.matmul(x, w1) + b1)

# Logit
w2 = tf.get_variable("W2", initializer=tf.contrib.layers.xavier_initializer(),
    shape=(self.input_size / 2, self.nb_targets))
b2 = tf.get_variable("b2", initializer=tf.contrib.layers.xavier_initializer(), shape=self.nb_targets)
self.logits = tf.matmul(x, w2) + b2

Il faut aussi penser normalement à ajouter les nouveaux poids crées à l'ensemble des valeurs entrainées, cependant la fonction ```tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)``` s'en charge automatiquement.

Les résultats ont été très différents entre chaque éxecution. Au début je n'obtenais pas d'excellents résultats avec une méthode classique d'optimisation tel que SGD, ou Momentum. Puis j'ai utilisé, à nouveau, la méthode d'optimisation d'Adam.  
Pour avoir essayer un MLP classique sur le problème MNIST, je me souvenais que les performances ne dépassait généralement pas 0.93 de précision test. Cependant je me suis rendu compte que la méthode d'Adam permettait d'obtenir des résultats très convenables (0.985 sur la précision test) avec un simple modèle à une couche cachée.  
De plus l'utilisation de la relu permet une convergence plus rapide que lorsque j'ai utilisé la sigmoid. Cela est du au fait qu'avec la dérivée de la sigmoid, l'intensité du signal du gradient est divisé par au moins un quart.  
Sur une des expériences testés, j'ai obtenues une précision test dépassant les 0.99, mais comme les initialisations de poids sont toujours aléatoires, je n'ai pas obtenue plusieurs fois ce résultat.  
Sur les courbes qui suivent, on peut cependant observer que le modèle sur-apprend __fortement__ sur les données d'entrainements. En effet, au bout d'une dizaine d'époque, la précision de training set est autour de 0.99, alors que la précision sur l'ensemble de validation ne dépasse pas 0.985. A la fin, la précision test est à peu près égale au dernière valeur de précision de l'ensemble de validation, c'est à dire 0.9812.  
Voici les différentes courbes:  
* Accuracy d'entrainement: ![Accuracy entrainement](images/nn_acc_train.png)
* Accuracy de validation: ![Accuracy entrainement](images/nn_acc_val.png)
* Accuracy de test: ![Accuracy entrainement](images/nn_acc_test.png)





## Question 3. Ajout d'une pénalité L2

__Code entier: code/l2regularization.py__

Les résultats précédents nous inscitent donc à ajouter une forme de régularisation au modèle. Nous allons commencer par une pénalité l2 des poids.  
__Ajouter une pénalité l2 aux poids revient à rajouter un à priori Gaussien sur les poids.__  
Des petites valeurs de coefficient l2 peuvent permettre de prévenir le sur apprentissage sur les données d'entrainements.  
La pénalité l2 de la forme $\alpha * \frac{1}{2}||W||_2^2$ est ajouté à la fonction de coût, où $\alpha$ est un hyper-paramètre.
Voici l'implémentation en Tensorflow:

In [ ]:
self.loss = # classique enthropie croisée

# self.l2_reg est un coefficient scalaire
self.loss += self.l2_reg *  tf.add_n([  # Somme sur toutes les variables
    (tf.reduce_sum(tf.square(v)) / 2) for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)# Voici formule au
                    if 'bias' not in v.name ]) # Il ne faut régularizer les bias

Voici donc ce qu'il faut rajouter à la fonction de coût, la fonction qui implémente.

Après plusieurs experimentations, voici les résultats pour l'ajout d'une contrainte l2 sur les poids.  
La contrainte l2 empêche le modèle de sur-apprendre sur les données d'entrainements, et cela se voit sur le graphe, puisque la précision d'entrainement ne dépasse 0.95. On observe par ailleurs, et c'est l'effet escompté, que la précision test/validation est au moins aussi bonne que la précision sur l'ensemble d'entrainement.  
Le modèle est contraint par les poids, et force ceux ci à être en moyenne nul, ce qui contraint l'optimisation à aller dans certaines régions de valeur de poids. Ces régions sont en général des régions où le modèle ajuste très bien l'ensemble d'entrainement.  
Ici, dans le cas de MNIST, la probabilité conditionnelle $p(y|x)$ est vraiment facile à ajuster pour un modèle à grande capacité comme un réseau de neurones. Cependant puisque les données ont été aussi très bien mélangées, et proviennet de la même distribution ($p(x_{entrainement}) = p(x_{test})$ à peu près), on a donc pas de problème de _covariance shift_, et il n'y a pas véritablement de différence entre les performances sur les différents ensembles.  
Ajouter une contrainte l2 fait donc perdre en précision.  
Résultat
* Accuracy d'entrainement: ![Accuracy entrainement](images/l2_acc_train.png)
* Accuracy de validation: ![Accuracy entrainement](images/l2_acc_val.png)
* Accuracy de test: ![Accuracy entrainement](images/l2_acc_test.png)


## Question 4. Transformation affine sur les données 

__Code entier: code/noisymodel.py__  
Les données vont être transformées de plusieurs manières.  
Voici les transformations effectuées:

In [ ]:
datagen = ImageDataGenerator(featurewise_center=True, # Centrer les données
                             featurewise_std_normalization=True, # Reduire les données
                             zca_whitening=True, # Decorreler les pixels
                             width_shift_range=0.2, # Shifter l'image en largeur
                             height_shift_range=0.2) # Shifter l'image en hauteur

Quelques modifications ont du être fait au code afin de fonctionner.

In [ ]:
datagen.fit(__reshape_mnist_image(mnist.train.images, True), augment=True)

# Cette fonction est appelé à toutes les itérations 
# d'une époque pour les données d'entrainements
def get_next_batch_training(batch_size):
    nb_batches = 0
    for x_batch, y_batch in datagen.flow(__reshape_mnist_image(mnist.train.images, True), mnist.train.labels,
                                         batch_size=batch_size):
        if nb_batches == (len(mnist.train.images) // batch_size):
            break
        nb_batches += 1
        yield __reshape_mnist_image(x_batch), y_batch

## Question 5. Recherche d'hyperparamètre

__Code entier: code/QTable.py, code/model.py code/state.py code/utils.py code/main_rl.py__

__Observations:__
* Avec 99.85 de précision test, nous avons déjà un bon modèle.  
* Nous n'avons utilisé aucune couche convolutionnelle
* 1 seule couche utilisé jusqu'à présent
* Aucune technique de normalisation intra-couches employées.

En se basant sur ce constat, on peut donc réfléchir à des moyens d'obtenir une meilleur performance. Notre baseline sera donc 99.85 de précision sur l'ensemble de test.  
J'ai privilégié la recherche d'une architecture optimale, car il est connu que 0.001 est un bon taux d'apprentissage pour Adam. D'une autre part, je n'ai pas ré-utilisé la pénalisation des poids, car j'ai utilisé la méthode de dropout couplé avec la normalisation par batches intra-couches

### Reinforcement Learning
Afin d'apprendre une nouvelle chose, et puisque j'avais déjà essayé Spearmint au dernier TP, j'ai décidé d'utiliser une technique de reinforcement learning, et plus précisiment l'algorithme Q-Learning afin de trouver une architecture optimale pour le problème. Je me suis donc basé sur un papier récent [Designing neural network architecture using reinforcement learning](https://arxiv.org/pdf/1611.02167.pdf). La différence notable avec le papier est que j'ai limité mon modèle à 4 couches, et ait contraint les couches à avoir des dimensions beaucoup plus petite que dans le papier.  

### Mise en place de l'algorithme
Le principe de l'algorithme est simple. Il s'agit de trouver une table de transition qui étant donné une couche d'un réseau de neurones renvoit la couche connécté suivante la plus optimale étant donné la précédente. Le problème peut se voir comme un problème de Markov où il faut trouver $p(l_t|l_{t-1})$.  
Afin de simplifier le problème, le nombre de couches différentes est finie, et surtout restreints. Voici les différentes combinaisons possibles:  
* Couche convolutionnel:


Paramètres | Valeurs prises
--- | --- 
Taille du filtre  | {[(1, 1), (2, 2)]} 
Stride | {1}
Nombre de channels | {32, 64, 128}

* Couche de pooling

Paramètres | Valeurs prises
--- | --- 
Pair (taille du kernel-stride)  | {[(3, 2), (2, 2), (2, 1)]} 

* Couche complètement connecté

Paramètres | Valeurs prises
--- | --- 
Nb couches FC successives | {1, 2}
Dimension de la couche  | {[128, 64, 32]}

Après avoir défini les différentes états possibles de notre système markovien (toutes les combinaisons de paramètres de chaque couche), il faut définir des probabilités de passer d'un état à un autre. Certains états ne peuvent transitionner d'un état à un autre, dans un pur soucis de complexité de calcul, mais avec plus de puissance de calcul, cela aurait été possible d'élargir la matrice de transition.   
Chaque probabilité de transition d'un état à un autre se voit affecter un probabilité initiale de 0.5.  
Voici quelques heuristiques sur les transitions:
* Les Fully-connected seulement à la fin
* Impossible de connecter une couche convolutionnel de 128 channels à un fully connected (trop de paramètres)
* Le nombre de couches entre fully connected doit toujours diminué
* D'autres règles plus complexes (expliquées dans le papier), pour conserver une représentation des données. Par exemple si l'on ajoute une couche de pooling avec un stride de 2, cela diminue par 2 la taille de chaque channel, diminuant aussi la représentation des données au sein de la couche. Il faut donc interdire d'arriver à une représentation trop petite (channel de taille 2x2 par exemple)
* Tout état à une probabilité initiale de 0.5 de finir, c'est à dire de ne pas rajouter une couche supplémentaire
* Au bout de 4 couches, tout état va vers un état terminant

### Principe de l'algorithme
L'algorithme est relativement simple. Voici le pseudo code pour l'algorithme de Q-Learning avec replay.
![](images/q_learning.png)

Voici l'algorithme pour échantilloner un nouveau réseau. Au début de l'apprentissage de la table de transition, l'algorithme privilégie l'exploration, ainsi la couche suivante est échantillonée de manière aléatoire sur les possible couches suivantes. Au fur et à mesure, la couche suivante est choisie en se basant sur la table construite, c'est à dire on prend $argmax_{l_{t+1}}p(l_{l_{t+1}} | l_t)$ où $l_t$ représente un état).
![](images/sample_new.png)

Et voici l'algorithme pour mettre à jour la table. Dans quasiment tous les algorithmes de reinforcement learning, un agent apprend à interagir avec son environnement en étant guider par une récompense; ici la récompense est la valeur de la précision test du modèle crée à partir de l'architecture échantillonée.
![](images/update.png)

Le code peut être trouvé dans ...
Différence par rapport aux réseaux construits avant. On ajoute une couche de dropout toutes les trois couches, plus batch normalisation sur toutes les couches pour réduire le décalage de covariance entre chaque passe vers l'avant dans le réseau.  
Chaque réseau est éxécuté pendant 20 iterations.  
A partir de cela, j'enregistre les 10 meilleures performances, mais je ne vais entrainer en profondeur qu'un seul (celui qui à le mieux performer).

### Résultats au bout de 1200 itérations et __quelques dizaines d'heures d'entrainements__
Voici les 10 meilleures performances sur la précision test:
![](images/result_best_model.png)

Les noms ne sont pas très clairs. Il y a cependant un code couleur, Voici les prédictions sur l'ensemble de validation de ces tests:
![](images/res_acc_val.png)

Et voici le tableau des trois meilleures architectures:

Couleur | Rang | Architecture
--- | --- | ---
Orange  | 1 | FC(64)->FC(32)->FC(10)
Bleu  | 2 | MaxPool({2, 1})->FC(64)->FC(32)
Vert clair  | 1 | MaxPool({2, 1})->ConvLayer({stride=1, filtre=2, channel=64})->FC(32)

Il est étonnant de remarquer que le réseau simple, sans convolution est celui qui a performé le mieux. Cependant cela peut s'expliquer par plusieurs raisons, si le modèle est trop gros en mémoire, il ne peut s"éxécuter et reçoit par défault une récompense de 0. Cela a sans doute influencé l'apprentissage.

### Implementation du meilleur modèle
Entrainons à présent le meilleur modèle.  
_(cf code/best_model.py)_
